# Spark

### Introduction:

This time you will create the data.



### Step 1. Import the necessary libraries

In [1]:
import numpy as np
from datetime import datetime
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession , Row
from pyspark.sql.types import *
from pyspark.sql.functions import col , when , lit , udf ,to_timestamp , monotonically_increasing_id 
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable



In [2]:
spark_sql = SparkSession.builder \
    .master('local[*]') \
    .appName('learn-spark') \
    .getOrCreate()
spark = spark_sql.sparkContext

In [3]:
# data = [('dung',32) , ('hau',26) , ('khang',22) , ('quan',21)]
# col = ['name' , 'age']
# df = spark_sql.createDataFrame(data , schema = col)
# df.show()


data_err_1 = spark_sql.read.option("header",True).csv("error_data_part1.csv")
data_err_2 = spark_sql.read.option("header",True).csv("error_data_part2.csv")
data = data_err_1.union(data_err_2).distinct()
data.show()

# data  = [1,2,3,4,5]
# distData = spark.parallelize(data)
# distData.collect()
# distFile = spark.textFile("data.txt")
# print(type(distFile))
# distFile.collect()




+------+-------------+----+
|errors|   product_id|code|
+------+-------------+----+
|     4|  PRODUCT_145|EMAN|
|     3| PRODUCT_1614|BLOK|
|     5| PRODUCT_1611|BPAK|
|    48| PRODUCT_2582|BPAK|
|     2| PRODUCT_2706|FQCH|
|     3| PRODUCT_4705|EMAN|
|     3|PRODUCT_10791|FQCH|
|     1|PRODUCT_11140|BPAK|
|     1|PRODUCT_11489|BLOK|
|     1|PRODUCT_12142|FQCH|
|     5|PRODUCT_12510|FQCH|
|    60|PRODUCT_14230|BPAK|
|     1|PRODUCT_14160|BLOK|
|     2| PRODUCT_1017|BPAK|
|     1| PRODUCT_1408|BLOK|
|     1| PRODUCT_3569|FQCH|
|     1| PRODUCT_3900|FQCH|
|     1|PRODUCT_11608|BPAK|
|     2|PRODUCT_11697|BLOK|
|     1|PRODUCT_11934|FQCH|
+------+-------------+----+
only showing top 20 rows



In [7]:
# data.dtypes

# data.take(5)
# data.fillna(0).collect()
# data.first()
# data.count()
# data.head(10)
# data_err_1.select('errors').show()
# data.show(data.count(),False)
data= data.drop('Unnamed: 0')
data.show()

+------+-------------+----+
|errors|   product_id|code|
+------+-------------+----+
|     4|  PRODUCT_145|EMAN|
|     3| PRODUCT_1614|BLOK|
|     5| PRODUCT_1611|BPAK|
|    48| PRODUCT_2582|BPAK|
|     2| PRODUCT_2706|FQCH|
|     3| PRODUCT_4705|EMAN|
|     3|PRODUCT_10791|FQCH|
|     1|PRODUCT_11140|BPAK|
|     1|PRODUCT_11489|BLOK|
|     1|PRODUCT_12142|FQCH|
|     5|PRODUCT_12510|FQCH|
|    60|PRODUCT_14230|BPAK|
|     1|PRODUCT_14160|BLOK|
|     2| PRODUCT_1017|BPAK|
|     1| PRODUCT_1408|BLOK|
|     1| PRODUCT_3569|FQCH|
|     1| PRODUCT_3900|FQCH|
|     1|PRODUCT_11608|BPAK|
|     2|PRODUCT_11697|BLOK|
|     1|PRODUCT_11934|FQCH|
+------+-------------+----+
only showing top 20 rows



In [8]:
data =  data.withColumn("errors", data["errors"].cast("float"))
data.filter(data["errors"] >= 96).show()


+------+-------------+----+
|errors|   product_id|code|
+------+-------------+----+
|  96.0| PRODUCT_1771|BLOK|
|  96.0|PRODUCT_11622|FQCH|
|  96.0| PRODUCT_1346|FQCH|
|  96.0| PRODUCT_2281|EMAN|
|  96.0|  PRODUCT_444|EMAN|
|  96.0| PRODUCT_2025|FQCH|
| 104.0|  PRODUCT_151|EMAN|
|  96.0| PRODUCT_1980|BPAK|
|  96.0| PRODUCT_1535|EMAN|
|  96.0| PRODUCT_1984|BPAK|
|  96.0|PRODUCT_14111|BLOK|
|  96.0| PRODUCT_2424|BPAK|
|  96.0| PRODUCT_1455|BPAK|
|  99.0| PRODUCT_1934|EMAN|
|  96.0| PRODUCT_1514|FQCH|
|  96.0|  PRODUCT_959|BPAK|
|  96.0| PRODUCT_1346|BPAK|
|  96.0|  PRODUCT_628|FQCH|
|  96.0|  PRODUCT_626|BLOK|
|  96.0| PRODUCT_2707|FQCH|
+------+-------------+----+
only showing top 20 rows



In [9]:
data.sort("errors" , ascending = False).show()


+------+-------------+----+
|errors|   product_id|code|
+------+-------------+----+
| 111.0|PRODUCT_11413|FQCH|
| 111.0|PRODUCT_11413|EMAN|
| 104.0|  PRODUCT_151|EMAN|
|  99.0| PRODUCT_1934|EMAN|
|  99.0|PRODUCT_10971|BLOK|
|  97.0| PRODUCT_2415|BPAK|
|  96.0|  PRODUCT_444|EMAN|
|  96.0| PRODUCT_1514|FQCH|
|  96.0| PRODUCT_2707|FQCH|
|  96.0| PRODUCT_1984|BPAK|
|  96.0| PRODUCT_1346|BPAK|
|  96.0| PRODUCT_1535|EMAN|
|  96.0| PRODUCT_1455|BPAK|
|  96.0| PRODUCT_2281|EMAN|
|  96.0|PRODUCT_14111|BLOK|
|  96.0|PRODUCT_11622|FQCH|
|  96.0| PRODUCT_2025|FQCH|
|  96.0| PRODUCT_1980|BPAK|
|  96.0| PRODUCT_1346|FQCH|
|  96.0| PRODUCT_2424|BPAK|
+------+-------------+----+
only showing top 20 rows



In [10]:
data.groupBy('code').avg().show()

+----+------------------+
|code|       avg(errors)|
+----+------------------+
|BPAK| 15.06148867313916|
|BLOK|12.132132132132131|
|EMAN| 13.16969696969697|
|FQCH|12.716981132075471|
+----+------------------+



In [11]:
data.groupBy('code').count().show()

+----+-----+
|code|count|
+----+-----+
|BPAK|  309|
|BLOK|  333|
|EMAN|  330|
|FQCH|  318|
+----+-----+



In [12]:
data_test = data.select('code').take(10)
data_test

[Row(code='EMAN'),
 Row(code='BLOK'),
 Row(code='BPAK'),
 Row(code='BPAK'),
 Row(code='FQCH'),
 Row(code='EMAN'),
 Row(code='FQCH'),
 Row(code='BPAK'),
 Row(code='BLOK'),
 Row(code='FQCH')]

In [13]:
for i in data_test :
    print(i.code)

EMAN
BLOK
BPAK
BPAK
FQCH
EMAN
FQCH
BPAK
BLOK
FQCH


In [14]:
data.where(col("code").isNotNull()).show()


+------+-------------+----+
|errors|   product_id|code|
+------+-------------+----+
|   4.0|  PRODUCT_145|EMAN|
|   3.0| PRODUCT_1614|BLOK|
|   5.0| PRODUCT_1611|BPAK|
|  48.0| PRODUCT_2582|BPAK|
|   2.0| PRODUCT_2706|FQCH|
|   3.0| PRODUCT_4705|EMAN|
|   3.0|PRODUCT_10791|FQCH|
|   1.0|PRODUCT_11140|BPAK|
|   1.0|PRODUCT_11489|BLOK|
|   1.0|PRODUCT_12142|FQCH|
|   5.0|PRODUCT_12510|FQCH|
|  60.0|PRODUCT_14230|BPAK|
|   1.0|PRODUCT_14160|BLOK|
|   2.0| PRODUCT_1017|BPAK|
|   1.0| PRODUCT_1408|BLOK|
|   1.0| PRODUCT_3569|FQCH|
|   1.0| PRODUCT_3900|FQCH|
|   1.0|PRODUCT_11608|BPAK|
|   2.0|PRODUCT_11697|BLOK|
|   1.0|PRODUCT_11934|FQCH|
+------+-------------+----+
only showing top 20 rows



In [15]:
data=data.withColumn("double_errors" , data['errors'] * 2)
data.show()

+------+-------------+----+-------------+
|errors|   product_id|code|double_errors|
+------+-------------+----+-------------+
|   4.0|  PRODUCT_145|EMAN|          8.0|
|   3.0| PRODUCT_1614|BLOK|          6.0|
|   5.0| PRODUCT_1611|BPAK|         10.0|
|  48.0| PRODUCT_2582|BPAK|         96.0|
|   2.0| PRODUCT_2706|FQCH|          4.0|
|   3.0| PRODUCT_4705|EMAN|          6.0|
|   3.0|PRODUCT_10791|FQCH|          6.0|
|   1.0|PRODUCT_11140|BPAK|          2.0|
|   1.0|PRODUCT_11489|BLOK|          2.0|
|   1.0|PRODUCT_12142|FQCH|          2.0|
|   5.0|PRODUCT_12510|FQCH|         10.0|
|  60.0|PRODUCT_14230|BPAK|        120.0|
|   1.0|PRODUCT_14160|BLOK|          2.0|
|   2.0| PRODUCT_1017|BPAK|          4.0|
|   1.0| PRODUCT_1408|BLOK|          2.0|
|   1.0| PRODUCT_3569|FQCH|          2.0|
|   1.0| PRODUCT_3900|FQCH|          2.0|
|   1.0|PRODUCT_11608|BPAK|          2.0|
|   2.0|PRODUCT_11697|BLOK|          4.0|
|   1.0|PRODUCT_11934|FQCH|          2.0|
+------+-------------+----+-------

In [16]:
data.withColumn("double_errors" , when(data['double_errors'] > 100 , data['double_errors']/2).otherwise(data.double_errors)).show()

+------+-------------+----+-------------+
|errors|   product_id|code|double_errors|
+------+-------------+----+-------------+
|   4.0|  PRODUCT_145|EMAN|          8.0|
|   3.0| PRODUCT_1614|BLOK|          6.0|
|   5.0| PRODUCT_1611|BPAK|         10.0|
|  48.0| PRODUCT_2582|BPAK|         96.0|
|   2.0| PRODUCT_2706|FQCH|          4.0|
|   3.0| PRODUCT_4705|EMAN|          6.0|
|   3.0|PRODUCT_10791|FQCH|          6.0|
|   1.0|PRODUCT_11140|BPAK|          2.0|
|   1.0|PRODUCT_11489|BLOK|          2.0|
|   1.0|PRODUCT_12142|FQCH|          2.0|
|   5.0|PRODUCT_12510|FQCH|         10.0|
|  60.0|PRODUCT_14230|BPAK|         60.0|
|   1.0|PRODUCT_14160|BLOK|          2.0|
|   2.0| PRODUCT_1017|BPAK|          4.0|
|   1.0| PRODUCT_1408|BLOK|          2.0|
|   1.0| PRODUCT_3569|FQCH|          2.0|
|   1.0| PRODUCT_3900|FQCH|          2.0|
|   1.0|PRODUCT_11608|BPAK|          2.0|
|   2.0|PRODUCT_11697|BLOK|          4.0|
|   1.0|PRODUCT_11934|FQCH|          2.0|
+------+-------------+----+-------

In [17]:
data.withColumn("double_errors" , when(data['double_errors'] == 2.0 , 0).otherwise(data.double_errors)).show()

+------+-------------+----+-------------+
|errors|   product_id|code|double_errors|
+------+-------------+----+-------------+
|   4.0|  PRODUCT_145|EMAN|          8.0|
|   3.0| PRODUCT_1614|BLOK|          6.0|
|   5.0| PRODUCT_1611|BPAK|         10.0|
|  48.0| PRODUCT_2582|BPAK|         96.0|
|   2.0| PRODUCT_2706|FQCH|          4.0|
|   3.0| PRODUCT_4705|EMAN|          6.0|
|   3.0|PRODUCT_10791|FQCH|          6.0|
|   1.0|PRODUCT_11140|BPAK|          0.0|
|   1.0|PRODUCT_11489|BLOK|          0.0|
|   1.0|PRODUCT_12142|FQCH|          0.0|
|   5.0|PRODUCT_12510|FQCH|         10.0|
|  60.0|PRODUCT_14230|BPAK|        120.0|
|   1.0|PRODUCT_14160|BLOK|          0.0|
|   2.0| PRODUCT_1017|BPAK|          4.0|
|   1.0| PRODUCT_1408|BLOK|          0.0|
|   1.0| PRODUCT_3569|FQCH|          0.0|
|   1.0| PRODUCT_3900|FQCH|          0.0|
|   1.0|PRODUCT_11608|BPAK|          0.0|
|   2.0|PRODUCT_11697|BLOK|          4.0|
|   1.0|PRODUCT_11934|FQCH|          0.0|
+------+-------------+----+-------

In [18]:
data = data.withColumn("isError" , lit(False))
data.show()

+------+-------------+----+-------------+-------+
|errors|   product_id|code|double_errors|isError|
+------+-------------+----+-------------+-------+
|   4.0|  PRODUCT_145|EMAN|          8.0|  false|
|   3.0| PRODUCT_1614|BLOK|          6.0|  false|
|   5.0| PRODUCT_1611|BPAK|         10.0|  false|
|  48.0| PRODUCT_2582|BPAK|         96.0|  false|
|   2.0| PRODUCT_2706|FQCH|          4.0|  false|
|   3.0| PRODUCT_4705|EMAN|          6.0|  false|
|   3.0|PRODUCT_10791|FQCH|          6.0|  false|
|   1.0|PRODUCT_11140|BPAK|          2.0|  false|
|   1.0|PRODUCT_11489|BLOK|          2.0|  false|
|   1.0|PRODUCT_12142|FQCH|          2.0|  false|
|   5.0|PRODUCT_12510|FQCH|         10.0|  false|
|  60.0|PRODUCT_14230|BPAK|        120.0|  false|
|   1.0|PRODUCT_14160|BLOK|          2.0|  false|
|   2.0| PRODUCT_1017|BPAK|          4.0|  false|
|   1.0| PRODUCT_1408|BLOK|          2.0|  false|
|   1.0| PRODUCT_3569|FQCH|          2.0|  false|
|   1.0| PRODUCT_3900|FQCH|          2.0|  false|


In [19]:
data_product_1 = spark_sql.read.option("header",True).csv("product_raw_data_part1.csv")
data_product_2 = spark_sql.read.option("header",True).csv("product_raw_data_part2.csv")
data_product = data_product_1.union(data_product_2).distinct()
data_product.show()


+----+------------+-------------------+-------------------+-------------+
| _c0|machine_name|            in_time|           out_time|   product_id|
+----+------------+-------------------+-------------------+-------------+
| 134| MACHINE_002|2014-12-08 03-48-14|2014-12-08 04-45-52|PRODUCT_10134|
| 732| MACHINE_002|2015-01-01 15-16-18|2015-01-01 16-19-21|PRODUCT_10732|
| 825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|PRODUCT_10825|
|1212| MACHINE_002|2015-01-21 17-46-07|2015-01-21 18-36-21|PRODUCT_11212|
|1303| MACHINE_002|2015-01-25 02-56-31|2015-01-25 03-41-39|PRODUCT_11303|
|1541| MACHINE_002|2015-02-04 09-00-25|2015-02-04 10-00-20|PRODUCT_11541|
|2255| MACHINE_002|2015-03-08 23-48-55|2015-03-09 01-03-15|PRODUCT_12255|
|2409| MACHINE_002|2015-03-14 16-02-40|2015-03-14 16-41-51|PRODUCT_12409|
|2440| MACHINE_002|2015-03-15 15-20-39|2015-03-15 16-00-45|PRODUCT_12440|
|3159| MACHINE_002|2015-04-17 02-42-38|2015-04-17 03-16-50|PRODUCT_13159|
|3881| MACHINE_002|2015-05-21 12-29-40

In [20]:
data = data_product.join(data , ['product_id'])

In [21]:
data.show()

+-------------+----+------------+-------------------+-------------------+------+----+-------------+-------+
|   product_id| _c0|machine_name|            in_time|           out_time|errors|code|double_errors|isError|
+-------------+----+------------+-------------------+-------------------+------+----+-------------+-------+
|PRODUCT_10825| 825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|   4.0|EMAN|          8.0|  false|
|PRODUCT_10222| 222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|   1.0|BPAK|          2.0|  false|
|PRODUCT_13980|3980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|   1.0|BLOK|          2.0|  false|
|PRODUCT_14257|4257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|  60.0|FQCH|        120.0|  false|
| PRODUCT_1947|1946| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|  51.0|BLOK|        102.0|  false|
| PRODUCT_2068|2067| MACHINE_001|2015-03-06 06-31-28|2015-03-06 05-46-03|   1.0|BLOK|          2.0|  false|
| PRODUCT_2797|2796| MACHINE

In [22]:
data = data.drop('_c0')

In [23]:
data = data.withColumnRenamed('in_time' , 'start_at') \
    .withColumnRenamed('out_time' , 'end_at') \
    .withColumnRenamed('errors' , 'errors_quantity') \
    .withColumnRenamed('double_errors' , 'double_errors_quantity') 


In [24]:
data.show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|  false|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|  false|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|  false|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|  false|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|  false|
| PRODUCT_2068| MACHINE_001|2015

In [25]:
data.filter(col('code').isNotNull()).show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|  false|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|  false|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|  false|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|  false|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|  false|
| PRODUCT_2068| MACHINE_001|2015

In [26]:
# data = data.drop('Unnamed: 0')
data.show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|  false|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|  false|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|  false|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|  false|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|  false|
| PRODUCT_2068| MACHINE_001|2015

In [27]:
data.replace(False , True , 'isError').show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|   true|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|   true|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|   true|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|   true|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|   true|
| PRODUCT_2068| MACHINE_001|2015

In [28]:
data = data.withColumn('isError' , when(col('errors_quantity') >= 96 , True).otherwise(col('isError')))

In [29]:
data.show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|  false|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|  false|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|  false|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|  false|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|  false|
| PRODUCT_2068| MACHINE_001|2015

In [30]:
array_isError_value = [ i.isError for i in data.select('isError').collect()]
print("Defective product number : {}".format(sum(array_isError_value)))

Defective product number : 54


In [31]:
data_des = data.describe()

In [32]:
print(data_des.collect()[1].errors_quantity)
data_des.show()

13.121759622937942
+-------+-------------+------------+-------------------+-------------------+------------------+----+----------------------+
|summary|   product_id|machine_name|           start_at|             end_at|   errors_quantity|code|double_errors_quantity|
+-------+-------------+------------+-------------------+-------------------+------------------+----+----------------------+
|  count|         1273|        1273|               1273|               1273|              1273|1273|                  1273|
|   mean|         null|        null|               null|               null|13.121759622937942|null|    26.243519245875884|
| stddev|         null|        null|               null|               null| 23.91609624906151|null|     47.83219249812302|
|    min|PRODUCT_10015| MACHINE_001|2014-12-02 03-31-48|2014-12-02 02-11-04|               1.0|BLOK|                   2.0|
|    max|  PRODUCT_989| MACHINE_002|2015-07-11 03-23-29|2015-07-11 05-08-32|             111.0|FQCH|             

In [33]:
data.withColumn('isError' , when(col('product_id') == 'PRODUCT_10825' , True).otherwise(col('isError'))).show()

+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|   product_id|machine_name|           start_at|             end_at|errors_quantity|code|double_errors_quantity|isError|
+-------------+------------+-------------------+-------------------+---------------+----+----------------------+-------+
|PRODUCT_10825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|            4.0|EMAN|                   8.0|   true|
|PRODUCT_10222| MACHINE_002|2014-12-12 08-22-48|2014-12-12 09-57-41|            1.0|BPAK|                   2.0|  false|
|PRODUCT_13980| MACHINE_002|2015-05-26 10-19-20|2015-05-26 11-05-50|            1.0|BLOK|                   2.0|  false|
|PRODUCT_14257| MACHINE_002|2015-06-11 08-56-01|2015-06-11 10-36-17|           60.0|FQCH|                 120.0|  false|
| PRODUCT_1947| MACHINE_001|2015-03-01 20-54-49|2015-03-01 18-59-29|           51.0|BLOK|                 102.0|  false|
| PRODUCT_2068| MACHINE_001|2015

In [34]:
data_product.show()

+----+------------+-------------------+-------------------+-------------+
| _c0|machine_name|            in_time|           out_time|   product_id|
+----+------------+-------------------+-------------------+-------------+
| 134| MACHINE_002|2014-12-08 03-48-14|2014-12-08 04-45-52|PRODUCT_10134|
| 732| MACHINE_002|2015-01-01 15-16-18|2015-01-01 16-19-21|PRODUCT_10732|
| 825| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|PRODUCT_10825|
|1212| MACHINE_002|2015-01-21 17-46-07|2015-01-21 18-36-21|PRODUCT_11212|
|1303| MACHINE_002|2015-01-25 02-56-31|2015-01-25 03-41-39|PRODUCT_11303|
|1541| MACHINE_002|2015-02-04 09-00-25|2015-02-04 10-00-20|PRODUCT_11541|
|2255| MACHINE_002|2015-03-08 23-48-55|2015-03-09 01-03-15|PRODUCT_12255|
|2409| MACHINE_002|2015-03-14 16-02-40|2015-03-14 16-41-51|PRODUCT_12409|
|2440| MACHINE_002|2015-03-15 15-20-39|2015-03-15 16-00-45|PRODUCT_12440|
|3159| MACHINE_002|2015-04-17 02-42-38|2015-04-17 03-16-50|PRODUCT_13159|
|3881| MACHINE_002|2015-05-21 12-29-40

In [35]:
data_product = data_product.drop('_c0')
data_product.show()

+------------+-------------------+-------------------+-------------+
|machine_name|            in_time|           out_time|   product_id|
+------------+-------------------+-------------------+-------------+
| MACHINE_002|2014-12-08 03-48-14|2014-12-08 04-45-52|PRODUCT_10134|
| MACHINE_002|2015-01-01 15-16-18|2015-01-01 16-19-21|PRODUCT_10732|
| MACHINE_002|2014-12-30 11-46-40|2014-12-30 16-22-10|PRODUCT_10825|
| MACHINE_002|2015-01-21 17-46-07|2015-01-21 18-36-21|PRODUCT_11212|
| MACHINE_002|2015-01-25 02-56-31|2015-01-25 03-41-39|PRODUCT_11303|
| MACHINE_002|2015-02-04 09-00-25|2015-02-04 10-00-20|PRODUCT_11541|
| MACHINE_002|2015-03-08 23-48-55|2015-03-09 01-03-15|PRODUCT_12255|
| MACHINE_002|2015-03-14 16-02-40|2015-03-14 16-41-51|PRODUCT_12409|
| MACHINE_002|2015-03-15 15-20-39|2015-03-15 16-00-45|PRODUCT_12440|
| MACHINE_002|2015-04-17 02-42-38|2015-04-17 03-16-50|PRODUCT_13159|
| MACHINE_002|2015-05-21 12-29-40|2015-05-21 14-56-06|PRODUCT_13881|
| MACHINE_002|2015-06-10 04-09-05|

In [36]:
data_product = data_product.select('machine_name', 'product_id' , \
                    to_timestamp(data_product['in_time'],'yyyy-MM-dd HH-mm-ss').alias('start_at'), \
                    to_timestamp(data_product['out_time'],'yyyy-MM-dd HH-mm-ss').alias('end_at'))
data_product.show()

+------------+-------------+-------------------+-------------------+
|machine_name|   product_id|           start_at|             end_at|
+------------+-------------+-------------------+-------------------+
| MACHINE_002|PRODUCT_10134|2014-12-08 03:48:14|2014-12-08 04:45:52|
| MACHINE_002|PRODUCT_10732|2015-01-01 15:16:18|2015-01-01 16:19:21|
| MACHINE_002|PRODUCT_10825|2014-12-30 11:46:40|2014-12-30 16:22:10|
| MACHINE_002|PRODUCT_11212|2015-01-21 17:46:07|2015-01-21 18:36:21|
| MACHINE_002|PRODUCT_11303|2015-01-25 02:56:31|2015-01-25 03:41:39|
| MACHINE_002|PRODUCT_11541|2015-02-04 09:00:25|2015-02-04 10:00:20|
| MACHINE_002|PRODUCT_12255|2015-03-08 23:48:55|2015-03-09 01:03:15|
| MACHINE_002|PRODUCT_12409|2015-03-14 16:02:40|2015-03-14 16:41:51|
| MACHINE_002|PRODUCT_12440|2015-03-15 15:20:39|2015-03-15 16:00:45|
| MACHINE_002|PRODUCT_13159|2015-04-17 02:42:38|2015-04-17 03:16:50|
| MACHINE_002|PRODUCT_13881|2015-05-21 12:29:40|2015-05-21 14:56:06|
| MACHINE_002|PRODUCT_14239|2015-0

In [37]:
data_product.printSchema()

root
 |-- machine_name: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- start_at: timestamp (nullable = true)
 |-- end_at: timestamp (nullable = true)



In [39]:

data_product.registerTempTable("data_product_1")
data_product.registerTempTable("data_product_2")

In [43]:
spark_sql.sql("SELECT dp1.machine_name ,dp1.product_id , dp1.start_at , dp1.end_at FROM data_product_1 as dp1 join data_product_2 as dp2 on dp1.product_id = dp2.product_id").show()

+------------+-------------+-------------------+-------------------+
|machine_name|   product_id|           start_at|             end_at|
+------------+-------------+-------------------+-------------------+
| MACHINE_002|PRODUCT_10134|2014-12-08 03:48:14|2014-12-08 04:45:52|
| MACHINE_002|PRODUCT_10732|2015-01-01 15:16:18|2015-01-01 16:19:21|
| MACHINE_002|PRODUCT_10825|2014-12-30 11:46:40|2014-12-30 16:22:10|
| MACHINE_002|PRODUCT_11212|2015-01-21 17:46:07|2015-01-21 18:36:21|
| MACHINE_002|PRODUCT_11303|2015-01-25 02:56:31|2015-01-25 03:41:39|
| MACHINE_002|PRODUCT_11541|2015-02-04 09:00:25|2015-02-04 10:00:20|
| MACHINE_002|PRODUCT_12255|2015-03-08 23:48:55|2015-03-09 01:03:15|
| MACHINE_002|PRODUCT_12409|2015-03-14 16:02:40|2015-03-14 16:41:51|
| MACHINE_002|PRODUCT_12440|2015-03-15 15:20:39|2015-03-15 16:00:45|
| MACHINE_002|PRODUCT_13159|2015-04-17 02:42:38|2015-04-17 03:16:50|
| MACHINE_002|PRODUCT_13881|2015-05-21 12:29:40|2015-05-21 14:56:06|
| MACHINE_002|PRODUCT_14239|2015-0

In [44]:
data_product.show()
data_product.printSchema()

+------------+-------------+-------------------+-------------------+------------+
|machine_name|   product_id|           start_at|             end_at|checkOverlap|
+------------+-------------+-------------------+-------------------+------------+
| MACHINE_002|PRODUCT_10010|2014-12-01 21:28:45|2014-12-01 22:02:35|           1|
| MACHINE_002|PRODUCT_10011|2014-12-01 22:02:43|2014-12-01 22:52:56|           1|
| MACHINE_002|PRODUCT_10012|2014-12-01 22:53:08|2014-12-01 23:39:59|           1|
| MACHINE_002|PRODUCT_10001|2014-12-01 23:40:12|2014-12-02 00:42:36|           1|
| MACHINE_002|PRODUCT_10000|2014-12-02 00:42:46|2014-12-02 01:47:57|           1|
| MACHINE_002|PRODUCT_10006|2014-12-02 01:48:07|2014-12-02 03:22:55|           1|
| MACHINE_002|PRODUCT_10005|2014-12-02 03:23:05|2014-12-02 04:40:40|           1|
| MACHINE_002|PRODUCT_10003|2014-12-02 04:40:50|2014-12-02 05:39:32|           1|
| MACHINE_002|PRODUCT_10007|2014-12-02 05:39:41|2014-12-02 06:58:17|           1|
| MACHINE_002|PR

In [50]:
udfYearExtract = udf(lambda x , y  : "{} - {}".format(datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').hour , datetime.strptime(str(y), '%Y-%m-%d %H:%M:%S').hour) , StringType())
data_product = data_product.withColumn("hours_done" , udfYearExtract("start_at" , "end_at"))
data_product.show()

+------------+-------------+-------------------+-------------------+------------+----------+
|machine_name|   product_id|           start_at|             end_at|checkOverlap|hours_done|
+------------+-------------+-------------------+-------------------+------------+----------+
| MACHINE_002|PRODUCT_10010|2014-12-01 21:28:45|2014-12-01 22:02:35|           1|   21 - 22|
| MACHINE_002|PRODUCT_10011|2014-12-01 22:02:43|2014-12-01 22:52:56|           1|   22 - 22|
| MACHINE_002|PRODUCT_10012|2014-12-01 22:53:08|2014-12-01 23:39:59|           1|   22 - 23|
| MACHINE_002|PRODUCT_10001|2014-12-01 23:40:12|2014-12-02 00:42:36|           1|    23 - 0|
| MACHINE_002|PRODUCT_10000|2014-12-02 00:42:46|2014-12-02 01:47:57|           1|     0 - 1|
| MACHINE_002|PRODUCT_10006|2014-12-02 01:48:07|2014-12-02 03:22:55|           1|     1 - 3|
| MACHINE_002|PRODUCT_10005|2014-12-02 03:23:05|2014-12-02 04:40:40|           1|     3 - 4|
| MACHINE_002|PRODUCT_10003|2014-12-02 04:40:50|2014-12-02 05:39:32|  